In [ ]:
using DifferentialEquations
using PyPlot

In [ ]:
model = @ode_def_nohes HastingsPowell begin
    dx = x * (1 - x) - a1 * x * y / (1 + b1 * x)
    dy = a1 * x * y / (1 + b1 * x) - a2 * y * z / (1 + b2 * y) - d1 * y
    dz = a2 * y * z / (1 + b2 * y) - d2 * z
end a1 = 5.0 a2 = 0.1 b1 => 3.0 b2 = 2.0 d1 = 0.4 d2 = 0.01 

In [ ]:
function speed_test(b1vals)
    prob = ODEProblem(model, [0.8, 0.2, 10.0], (0.0, 10000.0))
    
    for b1 in b1vals
        prob.f.b1 = b1
        bsol = solve(prob, CVODE_BDF(), reltol = 1e-8, abstol = 1e-8)        
    end
end

In [ ]:
@time bifts = speed_test(2.2:0.001:3.2);

In [ ]:
# I am not sure how to get back an Array of the solutions, as this is painfully slow
function speed_test2(b1vals)
    u0 = [0.8, 0.2, 10.0]
    tspan = (0.0, 10000.0)
    model = HastingsPowell()
    
    bsols = Array{ODESolution}(length(b1vals))
    for (i, b1) in enumerate(b1vals)
        model.b1 = b1
        prob = ODEProblem(model, u0, tspan)
        bsol = solve(prob, CVODE_BDF(), reltol = 1e-8, abstol = 1e-8)        
        bsols[i] = bsol
    end
    return bsols
end

In [ ]:
@time bifts = speed_test2(2.2:0.001:3.2);

In [ ]:
plot(bifts[1000].t, bifts[1000][1, :])
xlim(5000, 5500)

Try to use a comprehension to make the code even more like the Mathematica Table version

In [ ]:
function calc_sol(b1val)
    u0 = [0.8, 0.2, 10.0]
    tspan = (0.0, 10000.0)
    model = HastingsPowell()
    model.b1 = b1val
    return solve(ODEProblem(model, u0, tspan), CVODE_BDF(), reltol = 1e-8, abstol = 1e-8)
end

In [ ]:
b1vals = 2.2:0.001:3.2
@time bifts = [calc_sol(b1) for b1 in b1vals];

In [ ]:
plot(bifts[1000].t, bifts[1000][1, :])
xlim(5000, 5500)